# Loading Dataset

****The Multi-Genre Natural Language Inference (MNLI) corpus,
which is a collection of 392,702 sentence pairs annotated with entailment (contradic‐
tion, neutral, entailment). We will be using a subset of the data, 50,000 annotated
sentence pairs, to create a minimal example that does not need to be trained for hours
on end.****

In [2]:
from datasets import load_dataset
# Loading MNLI dataset from GLUE
# 0 = entailment, 1 = neutral, 2 = contradiction
train_dataset = load_dataset(
 "glue", "mnli", split="train"
).select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test_matched-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [3]:
print(train_dataset[3])
print(train_dataset[6])

{'premise': 'How do you know? All this is their information again.', 'hypothesis': 'This information belongs to them.', 'label': 0}
{'premise': 'But a few Christian mosaics survive above the apse is the Virgin with the infant Jesus, with the Archangel Gabriel to the right (his companion Michael, to the left, has vanished save for a few feathers from his wings).', 'hypothesis': 'Most of the Christian mosaics were destroyed by Muslims.  ', 'label': 1}


# Training the model 

**We typically choose an existing sentence-transformers model
and fine-tune that model, but in this example, we are going to train an embedding
from scratch.
This means that we will have to define two things. First, a pretrained Transformer
model that serves as embedding individual words. We will use the BERT base model
(uncased) as it is a great introduction model.**

In [4]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
# Using a base model
embedding_model = SentenceTransformer('bert-base-uncased')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.1 MB/s eta 0:00:00a 0:00:01


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Loss Function to optimize the model

In [5]:
from sentence_transformers import losses
#softmax loss
train_loss = losses.SoftmaxLoss(
 model=embedding_model,
 sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(), num_labels=3)


# Model Evaluation

**We can perform evaluation of the performance of our model using the Semantic
Textual Similarity Benchmark (STSB). It is a collection of human-labeled sentence
pairs, with similarity scores between 1 and 5.**

In [6]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
# Creating an embedding similarity evaluator for STSB
val_sts = load_dataset("glue", "stsb", split="validation")
evaluator = EmbeddingSimilarityEvaluator(
 sentences1=val_sts["sentence1"],
 sentences2=val_sts["sentence2"],
 scores=[score/5 for score in val_sts["label"]],
 main_similarity="cosine",
)

train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

# Defining the training arguements

**we have our evaluator, we create SentenceTransformerTrainingArgu
ments, similar to training with Hugging Face Transformers**

In [7]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

In [8]:
args = SentenceTransformerTrainingArguments(
 output_dir="base_embedding_model",
 num_train_epochs=1,
 per_device_train_batch_size=32,
 per_device_eval_batch_size=32,
 warmup_steps=100,
 fp16=True,
 eval_steps=100,
 logging_steps=100,
)

# Training the embedding model

In [9]:
from sentence_transformers.trainer import SentenceTransformerTrainer
# Training
trainer = SentenceTransformerTrainer(
 model=embedding_model,
 args=args,
 train_dataset=train_dataset,
 loss=train_loss,
 evaluator=evaluator
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,1.079800
200,0.934000
300,0.877300
400,0.841000
500,0.827900
600,0.823600
700,0.800200
800,0.789000
900,0.774800
1000,0.771800


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=1563, training_loss=0.8113608772146038, metrics={'train_runtime': 539.8233, 'train_samples_per_second': 92.623, 'train_steps_per_second': 2.895, 'total_flos': 0.0, 'train_loss': 0.8113608772146038, 'epoch': 1.0})

# Evaluating Results

In [14]:
# Evaluating our trained model
result=evaluator(embedding_model)
result

{'pearson_cosine': 0.5345397045887939,
 'spearman_cosine': 0.6082469138789448,
 'pearson_manhattan': 0.5801962729306847,
 'spearman_manhattan': 0.6068780695191375,
 'pearson_euclidean': 0.571048137696305,
 'spearman_euclidean': 0.6029259030941507,
 'pearson_dot': 0.5044776644612751,
 'spearman_dot': 0.5443529748240566,
 'pearson_max': 0.5801962729306847,
 'spearman_max': 0.6082469138789448}

In [16]:
print(result['pearson_cosine'])

0.5345397045887939
